# Detecting Discrepancies in User Signup and Click Locations

Understanding the consistency between user signup data and subsequent activity is crucial for detecting fraud, misuse, or unintended behavior. This notebook analyzes discrepancies between the **country where a user signed up** and the **country from which they are clicking**, which may indicate:

- Account takeovers
- VPN or proxy usage
- Fraudulent activities

In [1]:
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt

### Load CSV files
We will load the following:
- `7_1_clicks_country.csv`: Country code from where click was generated
- `7_2_clicks.csv`: Click-level info including `adv_id`
- `7_4_user_signup_location.csv`: Signup country of the user

In [14]:
# Load datasets
clicks_country = pd.read_csv('7_1_clicks_country.csv')
clicks = pd.read_csv('7_2_clicks.csv')
signup = pd.read_csv('7_4_user_signup_location.csv')
#signup

,adv_id,country_code
0,348d9cfa-5235-468f-9f6c-6d8c7f934119,IN
1,085c15df-a0f2-454a-a9d8-d50633f5d639,IN
2,9ffce855-e5f3-45f7-a90a-07f3b3c7f5ef,IN
3,0864553a-dda0-48c9-a605-2a30bb2a681b,IN
4,2f37dc38-570a-45ff-8e94-12f9e4e6fa24,IN
...,...,...
92557,be913e78-134e-4aa8-ac03-569d255030d2,NG
92558,757f2107-b8e3-4912-bc41-e7a97d51534f,ID
92559,80e4786e-1ff7-4029-b9b1-138c059de3bd,ID
92560,2a3123ce-60d2-4d6e-afdb-5da82d9614de,ID


## Step 2: Merge Datasets to Map Clicks with Signup Location
1. Join clicks with country info.
2. Merge with user signup info.

In [3]:
# Merge click_id with country
click_data = pd.merge(clicks, clicks_country, on='click_id', how='left')

# Merge with signup location
merged_data = pd.merge(click_data, signup, on='adv_id', how='left', suffixes=('_click', '_signup'))

## Step 3: Detect Discrepancies in Country Codes
We will filter users whose **click country** does not match their **signup country**.

In [4]:
# Filter mismatched country codes
discrepancies = merged_data[merged_data['country_code_click'] != merged_data['country_code_signup']]

## Step 4: Map Country Codes to Country Names
Use JSON from [PubScale Dashboard](https://dashboard.pubscale.com/json/country.json) to enrich with country names.

In [5]:
# Load country code-to-name mapping
country_url = "https://dashboard.pubscale.com/json/country.json"
country_mapping = requests.get(country_url).json()
country_df = pd.DataFrame(country_mapping)

# Map country codes to names
discrepancies['click_country_name'] = discrepancies['country_code_click'].map(country_df.set_index('code')['name'])
discrepancies['signup_country_name'] = discrepancies['country_code_signup'].map(country_df.set_index('code')['name'])

# Drop missing values after mapping (optional)
discrepancies.dropna(subset=['click_country_name', 'signup_country_name'], inplace=True)

## Step 5: View Top Discrepancies
List unique suspicious users with different signup and click countries.

In [6]:
# Get distinct mismatched users
suspicious_users = discrepancies[['adv_id', 'click_id', 'click_country_name', 'signup_country_name']].drop_duplicates()

# Show a sample
suspicious_users.head(10)

,adv_id,click_id,click_country_name,signup_country_name


In [7]:
# Check how many discrepancies exist
print("Total mismatches found:", discrepancies.shape[0])

# Preview a few rows
discrepancies[['adv_id', 'country_code_signup', 'country_code_click', 'signup_country_name', 'click_country_name']].head()

# Check for any NaNs in country name columns
discrepancies[['signup_country_name', 'click_country_name']].isnull().sum()

Total mismatches found: 0


signup_country_name    0
click_country_name     0
dtype: int64

## Step 6: Visualize Top Country Mismatches
We'll count the most common mismatched country flows using a barplot.

In [8]:
# Group by mismatched country pairs and count occurrences
country_pairs = (
    discrepancies
    .groupby(['signup_country_name', 'click_country_name'])
    .size()
    .reset_index(name='count')
)

# Filter top 10 country discrepancies
top_pairs = country_pairs.sort_values('count', ascending=False).head(10)

# Plot only if data is available
if not top_pairs.empty:
    plt.figure(figsize=(10,6))
    sns.barplot(x='count', y='signup_country_name', hue='click_country_name', data=top_pairs)
    plt.title("Top Country Discrepancies (Signup vs Click)")
    plt.xlabel("Count of Discrepancies")
    plt.ylabel("Signup Country")
    plt.legend(title="Click Country")
    plt.tight_layout()
    plt.show()
else:
    print("No mismatched country pairs found to visualize.")

No mismatched country pairs found to visualize.


## Summary

- We identified users whose **clicks originated from a different country** than where they signed up.
- This could point to **VPN usage**, **fraudulent activity**, or **suspicious access**.
- Such users can be flagged for **manual review**, **security verification**, or **automated alerts**.



**Extra Notes:**
- Analyze time gaps between signup and mismatched clicks.
- Incorporate login logs to validate DAUs with discrepancies.
- Score users by frequency/severity of mismatches.